# The Model

## setup

### input data

In [ ]:
## imports
#  modules
import numpy as np
import pandas as pd
import xarray as xr
import warnings
import os

# custom code
import dask_addons
from open_data import open_data_dir, data_dir_to_zarr
from neural_net import DNN, geoDataset
from open_fire_data import FlattenedTruthTable

warnings.filterwarnings('ignore')

2025-08-07 11:57:52,298 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


In [2]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 15.52 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61518,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:61543,Total threads: 2
Dashboard: http://127.0.0.1:61544/status,Memory: 3.88 GiB
Nanny: tcp://127.0.0.1:61521,


In [3]:
DATA_DIR_PATH = '../data/_ZARR_FILES'

data_path = os.path.join(DATA_DIR_PATH, 'alaska_full.zarr')
prior_data_path = os.path.join(DATA_DIR_PATH, 'alaska_prior.zarr')

data = xr.open_zarr(data_path, decode_timedelta=False)
prior_data = xr.open_zarr(prior_data_path, decode_timedelta=False)

In [4]:
input_ds = dask_addons.FlattenedDaskDataset(data, prior_data)
input_ds.setup()

[compute_running_totals] run time: 0.13564515113830566s


### fire data

In [ ]:
fire_data = FlattenedTruthTable(pd.read_csv('../data/_FIRE/alaska_range_csv/data.csv'))

In [ ]:
fire_data.data

### dataset

In [ ]:
ds = geoDataset(input_ds, fire_data, feature_num = 13)

## training


In [ ]:
import torch
from neural_net import train

model = DNN()
print(model)